# Selecting LLMs based on Context Length

Different LLMs have different context lengths. As a very immediate an practical example, OpenAI has two versions of GPT-3.5-Turbo: one with 4k context, another with 16k context. This notebook shows how to route between them based on input.

In [24]:
from langchain.prompts import PromptTemplate
from langchain.schema.prompt import PromptValue
from langchain.schema.messages import BaseMessage
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from typing import Union, Sequence

In [3]:
short_context_model = ChatOpenAI(model="gpt-3.5-turbo")
long_context_model = ChatOpenAI(model="gpt-3.5-turbo-16k")

In [4]:
def get_context_length(prompt: PromptValue):
    messages = prompt.to_messages()
    tokens = short_context_model.get_num_tokens_from_messages(messages)
    return tokens

In [5]:
prompt = PromptTemplate.from_template("Summarize this passage: {context}")

In [20]:
def choose_model(prompt: PromptValue):
    context_len = get_context_length(prompt)
    if context_len < 30:
        print("short model")
        return short_context_model
    else:
        print("long model")
        return long_context_model

In [25]:
chain = prompt | choose_model | StrOutputParser()

In [26]:
chain.invoke({"context": "a frog went to a pond"})

short model


'The passage mentions that a frog visited a pond.'

In [27]:
chain.invoke(
    {"context": "a frog went to a pond and sat on a log and went to a different pond"}
)

long model


'The passage describes a frog that moved from one pond to another and perched on a log.'